# **Instalar y cargar librerías necesarias**

In [10]:
# Instalamos la librería transformers si no está instalada
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
pip show transformers datasets

Name: transformers
Version: 4.47.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
---
Name: datasets
Version: 3.2.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests,

In [12]:
# Importamos las librerías necesarias
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
from google.colab import files
import json

In [24]:
# Cargar archivo desde la computadora
uploaded = files.upload()

# Identificar el nombre del archivo cargado
dataset_file = list(uploaded.keys())[0]

# Cargar el contenido del archivo JSON
with open(dataset_file, "r") as f:
    dataset = json.load(f)

print(f"Dataset cargado exitosamente desde {dataset_file}. Contiene {len(dataset)} entradas.")


Saving dataset.json to dataset (2).json
Dataset cargado exitosamente desde dataset (2).json. Contiene 70 entradas.


In [25]:
# Especificamos el modelo preentrenado que utilizaremos
model_name = "microsoft/DialoGPT-medium"

# Cargamos el modelo y el tokenizador asociados
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Modelo y tokenizador '{model_name}' cargados exitosamente.")

Modelo y tokenizador 'microsoft/DialoGPT-medium' cargados exitosamente.


In [27]:
# Configurar el token de padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [28]:
# Preparar los datos para el entrenamiento
train_data = []
for d in dataset:
    inputs = tokenizer(d["pregunta"], truncation=True, padding="max_length", max_length=32, return_tensors="pt")
    labels = tokenizer(d["respuesta"], truncation=True, padding="max_length", max_length=32, return_tensors="pt")

    train_data.append({
        "input_ids": inputs["input_ids"].squeeze(0),
        "attention_mask": inputs["attention_mask"].squeeze(0),
        "labels": labels["input_ids"].squeeze(0)
    })

In [30]:
# Crear una clase personalizada para el dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {key: val for key, val in self.data[idx].items()}

In [31]:
# Crear el dataset
train_dataset = CustomDataset(train_data)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs",
    save_steps=10,
    save_total_limit=2,
    report_to="none"
)

In [32]:
# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Iniciar el entrenamiento
print("Iniciando el entrenamiento...")
trainer.train()
print("Entrenamiento completado.")

Iniciando el entrenamiento...


Step,Training Loss


Entrenamiento completado.


In [33]:
# Guardamos el modelo y el tokenizador entrenados en una carpeta local
output_dir = "./mi-chatbot-tecnologico"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Modelo y tokenizador guardados en {output_dir}.")

Modelo y tokenizador guardados en ./mi-chatbot-tecnologico.
